# 문자 RNN으로 성씨 국적 분류하기

In [3]:
from argparse import Namespace

import os
import json

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

import tqdm

## `Vocabulary`, `Vectorizer`, `Dataset`

In [4]:
class Vocabulary(object):
    """ 매핑을 위해 텍스트 처리하고 어휘 사전 만드는 클래스 """
    
    def __init__(self, token_to_idx=None):
        """
        token_to_idx: 기존 토큰-인덱스 매핑 딕셔너리
        """
        if token_to_idx is None:
            token_to_idx = {}
        
        self._token_to_idx = token_to_idx
        
        self._idx_to_token = {idx:token for token, idx in 
                              self._token_to_idx.items()}
        
    def to_serializable(self):
        """ 직렬화 가능한 딕셔너리 반환 """
        return {'token_to_idx': self._token_to_idx}
    
    @classmethod
    def from_serializable(cls, contents):
        """ 직렬화된 딕셔너리에서 Vocabulary 객체 만듦 """
        return cls(**contents)
    
    def add_token(self, token):
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        
        return index
    
    def add_many(self, tokens):
        return [self.add_token(token) for token in tokens]
    
    def lookup_token(self, token):
        return self._token_to_idx[token]
    
    def lookup_index(self, index):
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
    
    # 클래스 자체의 내용 출력 메소드
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    
    def __len__(self):
        return len(self._token_to_idx)

In [5]:
# 성씨 문자열의 정수 매핑을 위한 SequenceVocabulary 클래스 
# unk_token, mask_token 등을 사용해서 처리함.

class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                mask_token="<MASK>", begin_seq_token="<BEGIN>",
                end_seq_token="<END>"):
        
        super(SequenceVocabulary, self).__init__(token_to_idx)
        
        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token
        
        self.mask_index = self.add_token(self._mask_token) # 0
        self.unk_index = self.add_token(self._unk_token) # 1
        self.begin_seq_index = self.add_token(self._begin_seq_token) # 2
        self.end_seq_index = self.add_token(self._end_seq_token) # 3
        
    def to_serializable(self):
        # 상속한 Vocabulary객체의 to_serializable() 결과 {'token_to_idx':{__}} 이어받음
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token':self._unk_token,
                        'mask_token':self._mask_token,
                        'begin_seq_token':self._begin_seq_token,
                        'end_seq_token':self._end_seq_token})
        return contents
    
    def lookup_token(self, token):
        if self.unk_index >= 0:
            # 해당 토큰이 없으면 unk_index 1 반환
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

In [6]:
class SurnameVectorizer(object):
    """ 어휘 사전 생성 및 관리 """
    
    def __init__(self, char_vocab, nationality_vocab):
        self.char_vocab = char_vocab
        self.nationality_vocab = nationality_vocab
        
    def vectorize(self, surname, vector_length=-1):
        indices = [self.char_vocab.begin_seq_index] # [2]
        indices.extend(self.char_vocab.lookup_token(token)
                      for token in surname) # [2, ...]
        indices.append(self.char_vocab.end_seq_index) # [2, ..., 3]
        
        if vector_length < 0:
            vector_length = len(indices) # 벡터 길이: 성씨 문자열 길이 + 2
            
        out_vector = np.zeros(vector_length, dtype=np.int64) 
        # ex. array([0, 0, 0, 0, 0], dtype=int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.char_vocab.mask_index # 남은 칸은 0 마스킹
        
        return out_vector, len(indices)
    
    @classmethod
    def from_dataframe(cls, surname_df):
        char_vocab = SequenceVocabulary()
        nationality_vocab = Vocabulary()
        
        for index, row in surname_df.iterrows():
            for char in row.surname:
                char_vocab.add_token(char)
            nationality_vocab.add_token(row.nationality)
            
        return cls(char_vocab, nationality_vocab)
    
    @classmethod
    def from_serializable(cls, contents):
        char_vocab = SequenceVocabulary.from_serializable(contents['char_vocab'])
        nat_vocab = Vocabulary.from_serializable(contents['nationality_vocab'])
        
        return cls(char_vocab=char_vocab, nationality_vocab=nat_vocab)
    
    def to_serializable(self):
        return {'char_vocab': self.char_vocab.to_serializable(),
               'nationality_vocab': self.nationality_vocab.to_serializable()}

In [7]:
class SurnameDataset(Dataset):
    def __init__(self, surname_df, vectorizer):
        self.surname_df = surname_df
        self._vectorizer = vectorizer
        
        self._max_seq_length = max(map(len, self.surname_df.surname)) + 2  # 19
        
        self.train_df = self.surname_df[self.surname_df.split=='train']
        self.train_size = len(self.train_df)  # 7680
        
        self.val_df = self.surname_df[self.surname_df.split=='val']
        self.validation_size = len(self.val_df)  # 1640
        
        self.test_df = self.surname_df[self.surname_df.split=='test']
        self.test_size = len(self.test_df)  # 1660
        
        self._lookup_dict = {'train': (self.train_df, self.train_size),
                            'val': (self.val_df, self.validation_size),
                            'test': (self.test_df, self.test_size)}
        
        self.set_split('train')
        
        # 클래스 가중치
        class_counts = self.train_df.nationality.value_counts().to_dict()
        
        def sort_key(item):
            return self._vectorizer.nationality_vocab.lookup_token(item[0]) # 알파벳 순 
        
        # 국가 알파벳 순으로 국가 빈도 정렬
        sorted_counts = sorted(class_counts.items(), key = sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)
#         tensor([0.0009, 0.0065, 0.0035, 0.0061, 0.0005, 0.0063, 0.0025, 0.0092, 0.0078,
#         0.0024, 0.0018, 0.0189, 0.0119, 0.0263, 0.0006, 0.0192, 0.0056, 0.0250])
        
        # 클래스 가중치 총합: tensor(0.1548)
        """ 
        클래스 범주 빈도 낮을수록, 높은 가중치
        빈도 높을수록, 낮은 가중치 ----> 왜 사용할까?
        """
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, surname_csv):
        surname_df = pd.read_csv(surname_csv)
        train_surname_df = surname_df[surname_df.split=='train']
        return cls(surname_df, SurnameVectorizer.from_dataframe(train_surname_df))
    
    @classmethod
    def load_dataset_and_load_vectorizer(cls, surname_csv, vectorizer_filepath):
        surname_df = pd.read_csv(surname_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(surname_df, vectorizer)
    
    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        with open(vectorizer_filepath) as fp:
            return SurnameVectorizer.from_serializable(json.load(fp))
        
    def save_vectorizer(self, vectorizer_filepath):
        with open(vectorizer_filepath, 'w') as fp:
            json.dump(self._vectorizer.to_serializable(), fp)
            
    def get_vectorizer(self):
        return self._vectorizer
    
    def set_split(self, split='train'):
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
    
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        row = self._target_df.iloc[index]
        
        surname_vector, vec_length = \
           self._vectorizer.vectorize(row.surname, self._max_seq_length)
        
        nationality_index = \
           self._vectorizer.nationality_vocab.lookup_token(row.nationality)
        
        return {'x_data':surname_vector,
               'y_target':nationality_index,
               'x_length':vec_length}
    
    def get_num_batches(self, batch_size):
        return len(self) // batch_size
    

def generate_batches(dataset, batch_size, shuffle=True, 
                     drop_last=True, device='cpu'):
    """
    DataLoader로 미니배치로 모은 후,
    CPU와 GPU 간 데이터를 간편하게 전환하는 제너레이터 
    - 각 텐서를 지정된 장치로 이동시킴.
    - 반환 타입: generator <_____>
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                           shuffle=shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

## 모델

In [8]:
def column_gather(y_out, x_lengths):
    """
    y_out에 있는 각 데이터포인트에서 마지막 벡터 추출
    
    배치 행 인덱스를 순회하며 x_lengths 
    값에 해당하는 인덱스 위치의 벡터 반환
    
    y_out: (batch, sequence, feature)
    x_lengths: (batch,)
    
    반환값 y_out: (batch, feature)
    """
    x_lengths = x_lengths.long().detach().cpu().numpy() - 1
    
    out = []
    for batch_index, column_index in enumerate(x_lengths):
        out.append(y_out[batch_index, column_index])
        
    return torch.stack(out)

class ElmanRNN(nn.Module):
    """ RNNCell 사용하여 만든 엘만 RNN 
    --> RNN 계산 명시적으로 드러내기 위함 """
    
    def __init__(self, input_size, hidden_size, batch_first=False):
        """
        input_size: 입력 벡터 크기
        hidden_size: 은닉 상태 벡터 크기 
        batch_first: 0번째 차원이 배치인지 여부 
        """
        super(ElmanRNN, self).__init__()
        
        self.rnn_cell = nn.RNNCell(input_size, hidden_size)
        # 입력-은닉 가중치 행렬, 은닉-은닉 가중치 행렬 만듦
        # 입력 벡터 행렬과 은닉 벡터 행렬을 받음
        
        self.batch_first = batch_first
        self.hidden_size = hidden_size
        
    def _initial_hidden(self, batch_size):
        return torch.zeros((batch_size, self.hidden_size))
    
    def forward(self, x_in, initial_hidden = None):
        """
        ElmanRNN의 정방향 계산
        
        x_in: 입력 데이터 텐서
        - 만약 batch_first==True면, (batch_size, seq_size, feature_size)
        - 아니면, (seq_size, batch_size, feature_size)
        initial_hidden: RNN의 초기 은닉 상태
        
        반환값 hiddens: 각 타임 스텝에서 RNN 출력 
        - 만약 batch_first==True면, (batch_size, seq_size, hidden_size)
        - 아니면, (seq_size, batch_size, hidden_size)
        """
        if self.batch_first:
            batch_size, seq_size, feat_size = x_in.size()
            x_in = x_in.permute(1,0,2)  # (seq_size, batch_size, feature_size) 형태로 돌려놓음
        else:
            seq_size, batch_size, feat_size = x_in.size()
        
        hiddens = []  # 각 타임스텝의 은닉벡터 결과행렬을 담을 리스트
        
        if initial_hidden is None: # 아직 진행이 되지 않았다면 
            initial_hidden = self._initial_hidden(batch_size)
            # (배치크기 * 은닉크기) 모양의 초기 은닉행렬 만듦
            initial_hidden = initial_hidden.to(x_in.device)
            """ 초기 은닉 상태 벡터는 모두 0 """
        hidden_t = initial_hidden
        
        """ 입력 벡터의 길이만큼 RNNCell 반복 """
        for t in range(seq_size):
            hidden_t = self.rnn_cell(x_in[t], hidden_t)
            """ 현재 타임 스텝의 은닉 벡터는 이전 타임 스텝의 은닉 벡터와 
            현재 입력 벡터를 가지고 만들어짐 """
            hiddens.append(hidden_t)
            """ 각 타임 스텝의 은닉 벡터 결과를 hiddens 리스트에 추가 """
        
        hiddens = torch.stack(hiddens)
        # 은닉 벡터 행렬을 모두 쌓아서 3차원 텐서를 만듦
        
        if self.batch_first:
            hiddens = hiddens.permute(1,0,2)
            # 0차원(seq_size)과 1차원(batch_size) 바꾸기
            # 또다시 (batch_size, seq_size, feat_size) 로 돌려놓음
        
        return hiddens
    
class SurnameClassifier(nn.Module):
    """ RNN으로 특성 추출하고 MLP로 분류하는 분류 모델 """
    
    def __init__(self, embedding_size, num_embeddings, num_classes,
                rnn_hidden_size, batch_first=True, padding_idx=0):
        """
        embedding_size: 문자 임베딩 크기
        num_embeddings: 임베딩할 문자 개수
        num_classes: 예측 벡터 크기 (국적 개수)
        rnn_hidden_size: RNN의 은닉 상태 크기
        batch_first: 입력 텐서의 0번째 차원이 배치인지 시퀀스인지
        padding_idx: 텐서 패딩을 위한 인덱스
        """
        super(SurnameClassifier, self).__init__()
        
        self.emb = nn.Embedding(num_embeddings=num_embeddings,
                               embedding_dim=embedding_size,
                               padding_idx=padding_idx)
        # classifier.emb: Embedding(80, 100, padding_idx=0)
        # - 타입: torch.nn.modules.sparse.Embedding
        
        self.rnn = ElmanRNN(input_size=embedding_size,
                           hidden_size=rnn_hidden_size,
                           batch_first=batch_first)
        # classifier.rnn : ElmanRNN( (rnn_cell): RNNCell(100, 64) )
        # - 타입: __main__.ElmanRNN
        
        self.fc1 = nn.Linear(in_features=rnn_hidden_size,
                            out_features=rnn_hidden_size)
        # classifier.fc1 : Linear(in_features=64, out_features=64, bias=True)
        # - 타입 : torch.nn.modules.linear.Linear
        
        self.fc2 = nn.Linear(in_features=rnn_hidden_size,
                             out_features=num_classes)
        # classifier.fc2 : Linear(in_features=64, out_features=18, bias=True)
        # - 타입 : torch.nn.modules.linear.Linear
        
    def forward(self, x_in, x_lengths=None, apply_softmax=False):
        """ 분류기의 정방향 계산
        
        x_in: (batch, input_dim) 크기
        x_lengths: 배치에 있는 각 시퀀스의 길이
        - 시퀀스의 마지막 벡터를 찾는데 사용
        
        반환 텐서: (batch, output_dim) 크기
        """
        x_embedded = self.emb(x_in)
        y_out = self.rnn(x_embedded)
        
        if x_lengths is not None:
            y_out = column_gather(y_out, x_lengths)
        else:
            y_out = y_out[:, -1, :] 
            # 각 시퀀스의 마지막 벡터 행렬
            # 최종 은닉 벡터 행렬
            
        y_out = F.relu(self.fc1(F.dropout(y_out, 0.5)))
        # 선형층1 --> 비선형 활성화 함수 relu
        
        y_out = self.fc2(F.dropout(y_out, 0.5))
        # --> 선형층2
        
        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)
            
        return y_out

In [9]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)
        
def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

## 설정

In [10]:
args = Namespace(\
    
    # 날짜와 경로 정보
    surname_csv = "data/surnames/surnames_with_splits.csv",
    vectorizer_file = "vectorizer.json",
    model_state_file = "model.pth",
    save_dir = "model_storage/ch6/surname_classification",
    
    # 모델 하이퍼파라미터
    char_embedding_size = 100,
    rnn_hidden_size = 64,
    
    # 훈련 하이퍼파라미터
    num_epochs = 100,
    learning_rate = 1e-3,
    batch_size = 64,
    seed = 1337,
    early_stopping_criteria = 5,
    
    # 실행 옵션
    cuda = True,
    catch_keyboard_interrupt = True,
    reload_from_files = False,
    expand_filepaths_to_save_dir = True)


# CUDA 체크
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else 'cpu')

print("CUDA 사용여부: {}".format(args.cuda))

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                       args.vectorizer_file)
    
    args.model_state_file = os.path.join(args.save_dir, 
                                         args.model_state_file)

    
# 재현성을 위해 시드 설정
set_seed_everywhere(args.seed, args.cuda)

# 디렉토리 처리
handle_dirs(args.save_dir)

CUDA 사용여부: False


In [11]:
if args.reload_from_files and os.path.exists(args.vectorizer_file):
    # 체크포인트 로드 
    dataset = SurnameDataset.load_dataset_and_load_vectorizer(args.surname_csv,
                                                             args.vectorizer_file)
else:
    # 데이터셋과 vectorizer 로드
    dataset = SurnameDataset.load_dataset_and_make_vectorizer(args.surname_csv)
    dataset.save_vectorizer(args.vectorizer_file)
    
vectorizer = dataset.get_vectorizer()

classifier = SurnameClassifier(embedding_size = args.char_embedding_size, # 100 임베딩 차원(크기)
                              num_embeddings = len(vectorizer.char_vocab), # 80 임베딩 개수 (문자 개수)
                              num_classes = len(vectorizer.nationality_vocab), # 18 국가 클래스(범주)
                              rnn_hidden_size = args.rnn_hidden_size,  # 64 은닉 벡터 행렬 크기
                              padding_idx = vectorizer.char_vocab.mask_index) # 0 마스킹(패딩) 넘버

## 모델 훈련

In [31]:
def make_train_state(args):
    return {'stop early':False,
           'early_stopping_step':0,
           'early_stopping_best_val':1e8, # 100000000.0
           'learning_rate': args.learning_rate, # 1e-3 (0.001)
           'epoch_index':0,
           'train_loss':[], # 각 epoch의 훈련 손실 
           'train_acc':[], # 각 epoch의 훈련 정확도
           'val_loss':[],  # 각 epoch의 val 손실 
           'val_acc':[],   # 각 epoch의 val 정확도
           'test_loss':-1,
           'test_acc':-1,
           'model_filename': args.model_state_file} # "model.pth"

def update_train_state(args, model, train_state):
    """ 훈련 상태 업데이트
    
    콤포넌트:
    - 조기 종료: 과대 적합 방지
    - 모델 체크포인트: 더 나은 모델 저장
    
    param args: 메인 매개변수
    param model: 훈련할 모델
    param train_state: 훈련 상태를 담은 딕셔너리
    
    returns: 새로운 훈련 상태
    """
    
    # 적어도 한번 모델 저장
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False
        
    # 성능이 향상되면 모델 저장
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]
        
        # 손실이 나빠지면 
        if loss_t >= loss_tm1:
            # 조기 종료 단계 업데이트
            train_state['early_stopping_step'] += 1
            """ 
            이전 val_loss 보다 현재 val_loss가 더 크면 
            early_stopping_step을 1씩 증가시켜 주는데,
            이게 5가 넘어가면 stop_early 항목을 True로 바꿔줌
            --> 조기 종료
            """
        # 손실이 감소하면
        else:
            # 최상의 모델 저장 
            # 첨에 100000000.0 보다 작으면 해당 손실로 업데이트
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t
            
            # 조기 종료 단계 재설정
            train_state['early_stopping_step'] = 0
            """
            손실이 한번이라도 감소하게 되면 
            조기 종료 단계를 다시 0으로 초기화 시킴 
            --> 이후에 다시 5번 손실 증가돼야 조기 종료
            """
        # 조기 종료 여부 확인
        train_state['stop_early'] = \
           train_state['early_stopping_step'] >= args.early_stopping_criteria
        # 5 이상이면 조기 종료시킴
        
    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [32]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)

loss_func = nn.CrossEntropyLoss(dataset.class_weights) # 가중손실함수 
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                mode='min', factor=0.5,
                                                patience=1)

train_state = make_train_state(args) # 훈련 상태 딕셔너리 만듦

epoch_bar = tqdm.notebook.tqdm(desc='training routine',
                              total=args.num_epochs,
                              position=0) # 바깥 루프에 대한 position: 0

dataset.set_split('train')
train_bar = tqdm.notebook.tqdm(desc='split=train',
                              total=dataset.get_num_batches(args.batch_size),
                              position=1, # 안쪽 루프에 대한 position: 1
                              leave=True) # 루프 완료시 진행률 그대로 남김

dataset.set_split('val')
val_bar = tqdm.notebook.tqdm(desc='split=val',
                            total=dataset.get_num_batches(args.batch_size),
                            position=1,  # 안쪽 루프에 대한 position: 1
                            leave=True) # 루프 완료시 진행률 그대로 남김

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index # epoch 회차 설정
        
        # 훈련 세트에 대한 순회
        
        # 훈련 세트와 배치 제너레이터 준비, 손실과 정확도를 0으로 설정
        dataset.set_split('train')
        batch_generator = generate_batches(dataset,
                                          batch_size=args.batch_size,
                                          device=args.device)
        
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()  # 모델을 훈련 모드로 설정
        
        # 총 120회 훈련 데이터셋 미니배치 루프 돎 
        # 각 배치를 돌때마다 손실 계산, 가중치 업데이트 (1epoch당 120번)
        for batch_index, batch_dict in enumerate(batch_generator): # 0~119
            # 1. 그래디언트 0 초기화
            optimizer.zero_grad()
            
            # 2. 출력 계산
            y_pred = classifier(x_in=batch_dict['x_data'],
                               x_lengths=batch_dict['x_length'])
            
            # 3. 손실 계산
            loss = loss_func(y_pred, batch_dict['y_target'])
            
            running_loss += (loss.item() - running_loss) / (batch_index + 1)
            
            # 4. 손실을 사용해 그래디언트 계산
            loss.backward()
            
            # 5. 옵티마이저로 가중치 업데이트
            optimizer.step()
            
            # 정확도 계산
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            
            # 진행 상태 막대 업데이트
            train_bar.set_postfix(loss=running_loss, acc=running_acc, 
                                  epoch = epoch_index)
            train_bar.update()
            
        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)
        
        # 검증 세트에 대한 순회
        
        # 검증 세트와 배치 제너레이터 준비, 손실과 정확도를 0으로 설정
        dataset.set_split('val')
        batch_generator = generate_batches(dataset,
                                          batch_size=args.batch_size,
                                          device=args.device)
        
        running_loss = 0.
        running_acc = 0.
        classifier.eval()
        """ 모델을 평가 모드로 설정하여 역전파 끔 
        --> 편향되지 않은 모델의 성능을 얻는 목적으로만 사용 """
        
        for batch_index, batch_dict in enumerate(batch_generator):
            # 1. 출력 계산
            y_pred = classifier(x_in=batch_dict['x_data'],
                               x_lengths=batch_dict['x_length'])
            
            # 2. 손실 계산
            loss = loss_func(y_pred, batch_dict['y_target'])
            running_loss += (loss.item() - running_loss) / (batch_index + 1)
            
            # 3. 정확도 계산
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            # 평균 정확도? ex. 0.90 --> 0.05 / 2 (0.025 --> 0.925) 
            # 0.035 / 3 (0.011 --> 0.936)
            # 정확도 증감분 / 배치 진도 --> 를 더해줌 (점진적 증감)
            
            val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                                epoch=epoch_index)
            val_bar.update()
            
        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)
        
        train_state = update_train_state(args=args, model=classifier,
                                        train_state=train_state)
        
        scheduler.step(train_state['val_loss'][-1])
        
        # 각 epoch 끝날때 train_bar과 val_bar 상태를 0으로 초기화시켜줌
        train_bar.n = 0
        val_bar.n = 0
        
        # epoch_bar 업데이트
        epoch_bar.update()
        
        if train_state['stop_early']:
            break
            
except KeyboardInterrupt:
    print("반복 중지")

training routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/120 [00:00<?, ?it/s]

split=val:   0%|          | 0/25 [00:00<?, ?it/s]

train set 보다 validation set 에서 손실이 더 높고, 정확도가 더 낮음

## 모델 검증
### 테스트 세트 손실, 정확도 계산

In [66]:
classifier.load_state_dict(torch.load(train_state['model_filename']))
# 모델.load_state_dict(torch.load(저장한 모델 파일 경로))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # 출력을 계산합니다
    y_pred =  classifier(batch_dict['x_data'],
                         x_lengths=batch_dict['x_length'])
    
    # 손실을 계산합니다
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # 정확도를 계산합니다
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

print("테스트 손실: {};".format(train_state['test_loss']))
print("테스트 정확도: {}".format(train_state['test_acc']))

테스트 손실: 1.8592317199707034;
테스트 정확도: 45.00000000000001


### 추론

In [96]:
def predict_nationality(surname, classifier, vectorizer):
    vectorized_surname, vec_length = vectorizer.vectorize(surname)
    # ex. vectorizer.vectorize('Wan')
    # --> (array([ 2, 26,  7, 25,  3], dtype=int64), 5)
    vectorized_surname = torch.tensor(vectorized_surname).unsqueeze(dim=0)
    # ex. tensor([[ 2, 26,  7, 25,  3]])
    vec_length = torch.tensor([vec_length], dtype=torch.int64) # tensor([5])
    
    result = classifier(vectorized_surname, vec_length, apply_softmax=True)
#     tensor([[0.0389, 0.3525, 0.0098, 0.0061, 0.0325, 0.0023, 0.0091, 0.0007, 0.0112,
#          0.0013, 0.0432, 0.1932, 0.0135, 0.0018, 0.0235, 0.0087, 0.0025, 0.2493]],
#        grad_fn=<SoftmaxBackward>)
    probability_values, indices = result.max(dim=1)
#     torch.return_types.max(
#         values=tensor([0.4360], grad_fn=<MaxBackward0>),
#         indices=tensor([17]))
   
    """ 분류 결과가 매번 달라짐 주의!!!! """
    
    index = indices.item() # 17
    prob_value = probability_values.item() # 0.4360

    predicted_nationality = vectorizer.nationality_vocab.lookup_index(index)

    return {'nationality': predicted_nationality, 'probability': prob_value, 'surname': surname}

classifier = classifier.to("cpu")
for surname in ['McMahan', 'Nakamoto', 'Wan', 'Cho']:
    print(predict_nationality(surname, classifier, vectorizer))

{'nationality': 'Irish', 'probability': 0.5733259320259094, 'surname': 'McMahan'}
{'nationality': 'Japanese', 'probability': 0.8905038833618164, 'surname': 'Nakamoto'}
{'nationality': 'Chinese', 'probability': 0.3168955445289612, 'surname': 'Wan'}
{'nationality': 'Vietnamese', 'probability': 0.3496793806552887, 'surname': 'Cho'}


성씨 'Wan'이랑 'Cho'는 'Korean', 'Chinese', 'Vietnamese' 사이에서 헷갈리는 모양!!
- 국적 예측 결과가 매번 달라짐 !!
- 테스트셋의 정확도가 45% 밖에 되지 않은 것 감안!